# Task 2

## 1. Outline the Analysis Steps:
- We’ll import libraries, load the data, and visualize it.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
import warnings
# import tensorflow as tf
# from keras.layers import LSTM
current_dir = os.getcwd()
# Append the parent directory to sys.path
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
# ignore warrnings
warnings.filterwarnings('ignore')

In [2]:
from scripts.oil_price_data_extraction import WorldDataAnalysis

In [3]:
data_extracter =WorldDataAnalysis()

In [4]:
data_extracter.fetch_world_data()

Fetching world data...
Fetch complete. Data type: <class 'wbdata.client.DataFrame'>
World data fetched and filtered by date successfully.
        date  GDP Growth (%)  Inflation Rate (%)  Unemployment Rate (%)  \
0 2023-01-01        2.833509            5.635669               4.916572   
1 2022-01-01        3.238021            7.930929               5.260711   
2 2021-01-01        6.350090            3.475403               6.064446   
3 2020-01-01       -2.878323            1.905664               6.608805   
4 2019-01-01        2.675966            2.216776               5.598931   

  Exchange Rate (USD)  
0                None  
1                None  
2                None  
3                None  
4                None  
Data saved to 'world_data.csv'.


In [5]:
data_extracter.resample_to_daily()

World data resampled to daily frequency.
            GDP Growth (%)  Inflation Rate (%)  Unemployment Rate (%)  \
date                                                                    
1987-01-01        3.753409            5.710119                    NaN   
1987-01-02        3.753409            5.710119                    NaN   
1987-01-03        3.753409            5.710119                    NaN   
1987-01-04        3.753409            5.710119                    NaN   
1987-01-05        3.753409            5.710119                    NaN   

           Exchange Rate (USD)  
date                            
1987-01-01                None  
1987-01-02                None  
1987-01-03                None  
1987-01-04                None  
1987-01-05                None  


In [6]:
file_path ='../data/BrentOilPrices.csv'
data_extracter.load_oil_prices_from_csv(file_path=file_path)

Oil prices loaded successfully.
            Price
Date             
1987-05-20  18.63
1987-05-21  18.45
1987-05-22  18.55
1987-05-25  18.60
1987-05-26  18.63


In [7]:
data_extracter.merge_data()

Data merged successfully, maintaining only dates from oil prices data.
            GDP Growth (%)  Inflation Rate (%)  Unemployment Rate (%)  \
Date                                                                    
1987-05-20        3.753409            5.710119                    NaN   
1987-05-21        3.753409            5.710119                    NaN   
1987-05-22        3.753409            5.710119                    NaN   
1987-05-25        3.753409            5.710119                    NaN   
1987-05-26        3.753409            5.710119                    NaN   

           Exchange Rate (USD)  Price  
Date                                   
1987-05-20                None  18.63  
1987-05-21                None  18.45  
1987-05-22                None  18.55  
1987-05-25                None  18.60  
1987-05-26                None  18.63  
Merged data saved to 'merged_data.csv'.


In [8]:
merged_data =pd.read_csv('../data/merged_data.csv')
merged_data.head()

,Date,GDP Growth (%),Inflation Rate (%),Unemployment Rate (%),Exchange Rate (USD),Price
0,1987-05-20,3.753409,5.710119,NaN,NaN,18.63
1,1987-05-21,3.753409,5.710119,NaN,NaN,18.45
2,1987-05-22,3.753409,5.710119,NaN,NaN,18.55
3,1987-05-25,3.753409,5.710119,NaN,NaN,18.60
4,1987-05-26,3.753409,5.710119,NaN,NaN,18.63


In [9]:
merged_data.isnull().sum()

Date                        0
GDP Growth (%)              0
Inflation Rate (%)          0
Unemployment Rate (%)     925
Exchange Rate (USD)      9011
Price                       0
dtype: int64

In [10]:
# Check for missing values and data types
print(merged_data.info())
print(merged_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9011 entries, 0 to 9010
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   9011 non-null   object 
 1   GDP Growth (%)         9011 non-null   float64
 2   Inflation Rate (%)     9011 non-null   float64
 3   Unemployment Rate (%)  8086 non-null   float64
 4   Exchange Rate (USD)    0 non-null      float64
 5   Price                  9011 non-null   float64
dtypes: float64(5), object(1)
memory usage: 422.5+ KB
None
       GDP Growth (%)  Inflation Rate (%)  Unemployment Rate (%)  \
count     9011.000000         9011.000000            8086.000000   
mean         3.013317            4.782402               5.999068   
std          1.595337            2.448874               0.343139   
min         -2.878323            1.443857               5.133780   
25%          2.675966            2.860449               5.832310   
50%        

In [11]:
data =merged_data.drop(columns=['Exchange Rate (USD)'])
data.dropna(inplace=True)
data

,Date,GDP Growth (%),Inflation Rate (%),Unemployment Rate (%),Price
925,1991-01-02,1.210701,8.996939,5.133780,26.78
926,1991-01-03,1.210701,8.996939,5.133780,25.05
927,1991-01-04,1.210701,8.996939,5.133780,24.08
928,1991-01-07,1.210701,8.996939,5.133780,25.93
929,1991-01-08,1.210701,8.996939,5.133780,25.73
...,...,...,...,...,...
9006,2022-11-08,3.238021,7.930929,5.260711,96.85
9007,2022-11-09,3.238021,7.930929,5.260711,93.05
9008,2022-11-10,3.238021,7.930929,5.260711,94.25
9009,2022-11-11,3.238021,7.930929,5.260711,96.37
